In [115]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

!pip install mlxtend==0.23.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [116]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [117]:
# prompt: Get the set of product that has been purchased

products = set()
for col in df.columns:
    products.update(df[col].unique())
print(products)


{'Bagel', 'Diaper', 'Milk', 'Cheese', 'Bread', 'Meat', 'Pencil', 'Wine', 'Eggs', nan}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [118]:
#create an itemset based on the products

products = set()
for col in df.columns:
    products.update(df[col].unique())

encoded_first_row = []
for _, row in df.iterrows():
    transaction_dict = {product: (1 if product in row.values else 0) for product in products}
    encoded_first_row.append(transaction_dict)

encoded_first_row[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'Bagel': 0,
 'Diaper': 1,
 'Milk': 0,
 'Cheese': 1,
 'Bread': 1,
 'Meat': 1,
 'Pencil': 1,
 'Wine': 1,
 'Eggs': 1,
 nan: 0}

In [119]:
# encoding the feature
new_df = df.fillna('NaN')
datas = new_df.values.flatten()
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(datas.reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [120]:
# create new dataframe from the encoded features
df_encoded = pd.DataFrame(encoded_data, columns=encoder.categories_[0])

product_data = pd.DataFrame(0, index=df.index, columns=encoder.categories_[0])

for i, row in new_df.iterrows():
    for product in row:
        product_data.loc[i, product] = 1

# show the new dataframe
product_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,NaN,Pencil,Wine
0,0,1,1,1,1,1,0,0,1,1
1,0,1,1,1,0,1,1,0,1,1
2,0,0,1,0,1,1,1,1,0,1
3,0,0,1,0,1,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1,1


In [121]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
product_data.drop(columns='NaN', inplace=True)
product_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,0,1,1,1,1,1,0,1,1
1,0,1,1,1,0,1,1,1,1
2,0,0,1,0,1,1,1,0,1
3,0,0,1,0,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [123]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules
freq_products = apriori(product_data, min_support=0.2, use_colnames=True)
freq_products

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bread, Bagel)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [125]:
asc_rule = association_rules(freq_products, metric='confidence', min_threshold=0.6)
asc_rule.drop(columns=['zhangs_metric'], inplace=True)
asc_rule

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
2,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
3,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
4,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
5,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
6,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
8,"(Cheese, Eggs)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773
9,"(Cheese, Meat)",(Eggs),0.323810,0.438095,0.215873,0.666667,1.521739,0.074014,1.685714


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

Antecedent Support <br>
Antecedent support measures how often the antecedent (the left side of the rule) appears in the dataset, expressed as a proportion of all transactions. For example, in Rule 0 (Bagel → Bread), the antecedent support is 0.425397, indicating that Bagel is present in approximately 42.54% of all transactions. This metric helps assess the popularity of the antecedent on its own and provides a baseline for understanding the rule’s strength relative to the dataset.

Consequent Support <br>
Consequent support shows the proportion of transactions containing the consequent (the right side of the rule). For instance, in Rule 0, the consequent support is 0.504762, meaning that Bread appears in 50.48% of all transactions. This value helps evaluate how frequently the consequent occurs in the dataset overall, independent of the antecedent.

Support <br>
Support indicates the percentage of transactions that contain both the antecedent and the consequent. It is a direct measure of the co-occurrence of the two itemsets. For Rule 0, the support is 0.279365, meaning that Bagel and Bread appear together in 27.94% of all transactions. Higher support values suggest a stronger overall relationship but do not account for randomness or independence.

Confidence <br>
Confidence measures the likelihood of finding the consequent in transactions that contain the antecedent. For example, Rule 0 has a confidence of 0.656716, indicating that when Bagel is purchased, Bread is also purchased in 65.67% of those cases. This metric highlights the predictive power of the rule and is often used to filter strong associations.

Lift <br>
Lift evaluates the strength of the association by comparing the confidence of the rule to the independent occurrence of the consequent. For Rule 0, the lift is 1.301042, meaning that purchasing Bagel increases the likelihood of buying Bread by 30.1% compared to random chance. Lift values greater than 1 signify a positive association, while values close to or below 1 suggest weak or no association.

Leverage <br>
Leverage quantifies the difference between the observed support of the rule and the expected support if the antecedent and consequent were independent. For Rule 0, the leverage is 0.064641, indicating a small but positive deviation from independence. A leverage value close to zero implies little to no association, while higher values suggest a meaningful relationship.

Conviction <br>
Conviction measures the reliability of the rule by comparing the probability of the rule making a correct prediction against the likelihood of it being violated. In Rule 0, the conviction is 1.442650, suggesting that when Bagel is purchased, it is approximately 1.44 times more likely for Bread to also be purchased compared to random occurrence. Higher conviction values point to more robust and dependable rules.

By analyzing each of these metrics, we can interpret the strength and significance of the rules. For instance, rules with high lift, confidence, and leverage, such as those involving Eggs, Meat, and Cheese, indicate strong associations that can inform cross-selling and marketing strategies.

source: <br>
https://chatgpt.com/c/67482f90-1b74-8003-b865-31988a5d550e
https://medium.com/@kaanerdenn/association-rule-learning-arl-f2e6bf35bd98#:~:text=Consequents%3A%20Consequents%20are%20the%20items,the%20antecedent%20and%20consequent%20items.